In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize

## Wczytanie danych treningowych

In [2]:
traindf = pd.read_csv("train.csv")
traindf_complete = traindf.loc[:,['ora',
#                               'irr_pvgis_mod','irri_pvgis_mod',
                                 'temperatura_ambiente',
                                 'irradiamento',
                                 'pressure', 'windspeed',
                                 'cloudcover',
                                 'kwh']]

X = traindf_complete.drop(['kwh'], axis=1)
Y = traindf_complete['kwh']

X.shape

(235790, 6)

## //risercz algorytmów

In [3]:
# x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.4)

# y_test.head(10)

In [4]:
# %%time
## pierwsza proba
# # clf = svm.SVR(C=1, gamma=0.1, epsilon=0.1).fit(x_train, y_train)

## druga proba
# lss = Lasso(random_state=0, max_iter=1000)
# aaa = np.logspace(-10, -5, 30)
# par = [{'alpha': alphas}]
# nnn = 3

# clf = GridSearchCV(lss, par, cv=nnn, refit=True)
# clf.fit(x_train,y_train)

## trzecia proba
# clf = RandomForestRegressor(max_depth = 20, n_estimators=200, n_jobs=4)
# clf.fit(X, Y)

In [5]:
# clf.score(x_test, y_test)
# clf.best_score_

In [6]:
# clf.best_params_

In [7]:
# from sklearn.metrics import mean_squared_error
# yyy = clf.predict(x_test)
# mean_squared_error(y_test, yyy)

In [8]:
# yyy[:10]

## Wczytanie danych testowych

In [9]:
testdf = pd.read_csv("test.csv")
testdf_complete = testdf.loc[:,['ora',
#                                          'irr_pvgis_mod','irri_pvgis_mod',
                                'temperatura_ambiente',
                                 'irradiamento',
                                 'pressure', 'windspeed',
                                 'cloudcover'
                               ]]

testdf_complete.head(3)

,ora,temperatura_ambiente,irradiamento,pressure,windspeed,cloudcover
0,0.0,0.212,0.0,0.0,0.041,0.00
1,0.0,0.212,0.0,0.0,0.001,0.00
2,0.0,0.212,0.0,0.0,0.021,0.31


## Uczenie

In [10]:
%%time

modelfit = RandomForestRegressor(max_depth = 20, n_estimators=200, n_jobs=4, random_state=0)

modelfit.fit(X, Y)

Wall time: 57.2 s


## Predykcja

In [11]:
%%time
pred_result = modelfit.predict(testdf_complete)

Wall time: 594 ms


## Przygotowanie wyniku

In [14]:
non_negative = np.round(pred_result*1000)/1000.0
non_negative[non_negative < 0] = 0
d = {'Id': testdf['id'].tolist(), 'kwh': non_negative}
result = pd.DataFrame(data=d)

## Zapis do csv

In [15]:
result.to_csv('wynik.csv',index=False)